# Thématique 1 : l'eau (TP2 - analyse et interprétation des données)

Version A.

Version B.

Version C.

Version D.

L'analyse des données se fera en utilisant le langage de programmation Python.

## Les données des balances

Lors de l'expérience qui s'est déroulée en S2 et S3, 6 pots étaient posés sur des balances reliées à des "data loggers" (ou "collecteur de données") enregistrant le poids mesuré par chaque balance toutes les 10 minutes. Trois pots ne contenaient que du terreau. Ces pots "témoins" permettent d'observer ce qu'il se passe en l'absence de plante. Les 3 autres pots contenaient du terreau dans lequel était enraciné un plant de maïs.

Les données collectées se trouve dans un fichier [CSV](https://fr.wikipedia.org/wiki/Comma-separated_values) ("Comma-separated values") nommé ```balances-2025.csv``` et contenant 7 colonnes :
- ```time``` (l'instant à laquelle la mesure a été effectuée),
- ```tem_1```, ```tem_2```, et ```tem_3``` contenant les données des pots témoins, et
- ```plant_1```, ```plant_2``` et ```plant_3``` contenant les données des pots contenant un plant de maïs.

Pour manipuler les données, nous allons utiliser la bibliothèque Python [Pandas](https://pandas.pydata.org/docs/user_guide/10min.html). Pandas s'importe de la façon suivante, en utilisant par convention l'alias ```pd``` :

In [ ]:
import pandas as pd

Une fois Pandas importé, on lit le fichier CSV en utilisant la fonction ```read_csv()``` et en indiquant si nécessaire le caractère utilisé comme séparateur de colonnes dans le fichier CSV (typiquement la "," ou le ";") :

In [ ]:
# le paramètre optionnel 'sep' permet d'indiquer le caractère utilisé comme séparateur de colonnes, le point-virgule dans ce cas-ci
df = pd.read_csv('data/balances-2025.csv', sep=';')

La fonction ```read_csv()``` transforme le fichier CSV en un objet que l'on appelle un *DataFrame* (communément abrégé ```df```). Concrètement, il ne s'agit de rien de plus qu'un tableau à deux dimensions contenant des lignes et des colonnes, analogue à un tableur Excel.

La fonction ```df.head()``` permet d'afficher les 5 premières lignes d'un DataFrame.

In [ ]:
df.head()

*Remarque* : `df.head()` fonctionne tel quel dans la console de Spyder ou dans un Jupyter Notebook, mais dans un script Python vous devrez utiliser `print(df.head())` pour rendre les 5 lignes retournées par la fonction `head()` visibles.

Les 7 colonnes annoncées sont bien là ! La colonne ```time``` n'est cependant pas très lisible. Et pour cause, l'unité utilisée par les balances pour indiquer l'instant de la mesure est le nombre de jours écoulés depuis le 30 décembre 1899.

La fonction ```to_datetime()``` de Pandas permet de convertir ce genre de données en un objet Python spécifiquement destiné à stocker et manipuler des dates et des heures (datetime). On applique cette fonction à la colonne ```time``` et on remplace cette colonne par le résultat de l'opération :

In [ ]:
# les paramètres 'unit' et 'origin' permettent d'indiquer l'unité et l'origine des données temporelles à convertir en objets datetime
df['time'] = pd.to_datetime(df['time'], unit="d", origin="1899-12-30")

On utilise à nouveau la fonction ```df.head()``` pour afficher les 5 premières lignes du DataFrame et vérifier le résultat de l'opération :

In [ ]:
df.head()

En passant, on a vu que l'on accède à une colonne spécifique d'un DataFrame de la façon suivante `df['<nom de la colonne>']`. Ainsi, `df['plant_1']` contient les données concernant la plante n°1 :

In [ ]:
df['plant_1'].head()

De manière équivalente, il est possible d'accéder à la colonne ```plant_1``` via ```df.plant_1```, ce qui est parfois plus commode à écrire que `df['plant_1']` :

In [ ]:
df.plant_1.head()

Pour se simplifier la vie, on indique ensuite à Pandas que l'on souhaite utiliser la colonne ```time``` comme indices ("index") du DataFrame :

In [ ]:
df = df.set_index('time')
df.head()

De cette manière, il est possible de facilement sélectionner les données du tableau correspondantes à une période donnée, par exemple pour récupérer les lignes du tableau du 01-02-2025 au 02-02-2025 comme ci-dessous (notez qu'il faut indiquer la date en respectant le format ```YYYY-MM-JJ```) :

In [ ]:
df['2025-02-01':'2025-02-02']

Cette façon de sélectionner des données (ici des lignes du DataFrame) en indiquant un intervalle via la syntaxe `<début de l'intervalle>:<fin de l'intervalle>` entre `[ ]` s'appelle du "slicing" (de l'anglais "to slice" = "trancher"). Vous l'avez sûrement déjà utilisé par ailleurs pour récupérer des parties de listes.

*Remarque n°1* : contrairement à ce dont on a l'habitude en Python, avec Pandas, la fin de l'intervalle est incluse dans la tranche extraite du DataFrame (voir "[Endpoints are inclusive](https://pandas.pydata.org/docs/user_guide/advanced.html#endpoints-are-inclusive)"). Pour s'en convaincre, regardons les 5 dernières lignes du DataFrame quand on le "slice" avec l'intervalle `'2025-02-01':'2025-02-02'` :

In [ ]:
df['2025-02-01':'2025-02-02'].tail()

On voit que l'on récupère en effet entièrement la fin de l'intervalle indiqué, soit le 02-02-2025.

*Remarque n°2* : on a souvent besoin de sélectionner à la fois une colonne particulière (par exemple `plant_1`, via `df['plant_1']`) et un ensemble de lignes particulières de cette colonne (par exemple celles du 01-02-2025 au 01-02-2025, via le *slicing*). On pourrait alors être tenté de combiner les deux méthodes vues précédemment, par exemple comme ceci :

In [ ]:
df['plant_1']['2025-02-01':'2025-02-02']

Et effectivement, on voit que cela fonctionne. Néanmoins, si cela ne pose pas de problème quand il ne s'agit que d'*accéder* aux données, cela peut avoir des conséquences inattendues lorsqu'il s'agit de *modifier* les données sélectionnées (voir ["Chained Assignement"](https://pandas.pydata.org/docs/user_guide/copy_on_write.html#chained-assignment)). Pour éviter les problèmes et les longues heures de débogage, il vaut alors mieux utiliser `df.loc[]` comme ceci :

In [ ]:
df.loc['2025-02-01':'2025-02-01', 'plant_1']

En bref :
- pour sélectionner une colonne par son nom : `df[<nom de la colonne>]` ;
- pour sélectionner un intervalle de lignes ("slicing") : `df[<début de l'intervalle>:<fin de l'intervalle>]` ;
- pour faire les deux à la fois : `df.loc[<début de l'intervalle>:<fin de l'intervalle>, <nom de la colonne>]`.

En cas de doute, référez-vous aux exemples donnés dans cette introduction et n'hésitez pas à faire des tests dans la Console de Spyder.

Pour vous aider, nous vous guiderons encore pour les questions 1 et 3 ci-dessous.

### Question 1

Ajoutez une nouvelle colonne ```evap``` au DataFrame. Cette colonne doit contenir l'**évaporation** cumulée moyenne (en g d'eau) des 3 pots témoins.

#### Correction

Méthode n°1 (la plus simple) : on somme les 3 colonnes ```tem_i```, on divise le résultat par 3, et on prend l'opposé :

In [ ]:
df['evap'] = -(df['tem_1'] + df['tem_2'] + df['tem_3'])/3
df.head()

Méthode n°2 : on sélectionne les 3 colonnes ```tem_i``` et on obtient la moyenne via la fonction ```mean()``` en spécifiant ```axis=1``` pour que la moyenne soit calculée le long des lignes et non le long des colonnes :

In [ ]:
# Cette méthode à l'avantage d'illustrer la sélection multiple de colonnes, pas encore vue jusqu'ici
df['evap'] = -df[['tem_1', 'tem_2', 'tem_3']].mean(axis=1)
df.head()

Nous n'avons plus besoin des colonnes ```tem_i```. Pour simplifier l'affichage du DataFrame pour la suite, on peut supprimer ces colonnes avec la fonction ```drop()``` et en précisant les colonnes à supprimer :

In [ ]:
df = df.drop(columns=['tem_1', 'tem_2', 'tem_3'])
df.head()

### Question 2

Ajoutez 3 nouvelles colonnes `trans_1`, `trans_2` et `trans_3` au DataFrame. La colonne `trans_i` doit contenir la **transpiration** cumulée (en g d'eau) du plant de maïs $i$.

#### Correction

On prend l'opposée de la colonne ```plant_i``` et on soustrait l'évaporation moyenne pour passer de l'évapotranspiration à la transpiration :

In [ ]:
df['trans_1'] = -df['plant_1'] - df['evap']
df['trans_2'] = -df['plant_2'] - df['evap']
df['trans_3'] = -df['plant_3'] - df['evap']

Comme précédemment, on peut se débarrasser des colonnes ```plant_i``` :

In [ ]:
df = df.drop(columns=['plant_1', 'plant_2', 'plant_3'])
df.head()

### Question 3

Représentez graphiquement la transpiration cumulée des 3 plants de maïs ainsi que l'évaporation cumulée moyenne au cours du temps.

_Hint_ : `df.plot()`.

#### Correction

Il existe plein de façon de représenter graphiquement des données en Python, par exemple via les bibliothèques [Matplotlib](https://matplotlib.org/) et [Seaborn](https://seaborn.pydata.org/). Ici, comme l'objectif est plus d'explorer les données que de produire de magnifiques graphiques prêts à être publiés, on va se contenter de la fonction `plot()` de Pandas (qui utilise en réalité Matplotlib derrière). Celle-ci s'applique directement à un DataFrame, à une colonne spécifique d'un DataFrame ou à une portion quelconque de DataFrame.

In [ ]:
# On importe la bibliothèque Numpy de Python sous l'alias conventionnel "np". Cette bibliothèque est très utile, mais ici on ne l'utilise que de manière anecdotique pour légèrement améliorer l'affichage de notre graphique (voir la fonction np.unique() ci-dessous).
import numpy as np

# Le paramètre optionnel xticks permet de dire à Pandas que l'on veut que toutes les dates soient affichées sur l'axe des x. Essayez sans, vous verrez que c'est moins lisible.
df.plot(grid=True, xticks=np.unique(df.index.date), ylabel="[g d'eau]")

*Remarque* :
- Pandas affiche automatiquement les données en fonction du temps parce que nous avons définit la colonne `time` comme indices du DataFrame via la fonction `set_index()` plus haut. Sans cela, il aurait fallu préciser à Pandas qu'afficher sur l'axe des x ;
- sur le graphique ci-dessus, l'espace séparent deux lignes verticales correspond à une journée (24 heures), de 00:00 à 00:00.

### Question 4


Note pour les chercheur·ses impliqué·es : les étudiant·es répondent à l'une des 4 versions de la question ci-dessous selon leur groupe de TP.

Pour faciliter l'interprétation, représentez graphiquement la transpiration cumulée de la plante 1 sur une période plus courte, entre le 07-02-2025 et le 11-02-2025.


Pour faciliter l'interprétation, représentez graphiquement la transpiration cumulée de la plante 2 sur une période plus courte, entre le 08-02-2025 et le 12-02-2025.

Pour faciliter l'interprétation, représentez graphiquement la transpiration cumulée de la plante 3 sur une période plus courte, entre le 06-02-2025 et le 10-02-2025.

Pour faciliter l'interprétation, représentez graphiquement la transpiration cumulée de la plante 1 sur une période plus courte, entre le 06-02-2025 et le 9-02-2025.

#### Correction

On utilise à nouveau la fonction `plot()`, mais cette fois sur une sélection précise du DataFrame via `loc[]` :

In [ ]:
df_cropped = df.loc["2025-02-07":"2025-02-11", 'trans_1']

df_cropped.plot(grid=True, xticks=np.unique(df_cropped.index.date), ylabel="[g d'eau]")

### Question 5

Assez de manipulation de données, passons à leur interpretation : comment évoluent la transpiration et l'évaporation au cours d'une journée ? Et sur plusieurs jours ?

## Les données des encodeurs rotatifs

Pour mesurer la croissance d'une feuille d'un plant de maïs, son extrémité est attachée à une ficelle. Cette ficelle passe par une poulie accrochée au-dessus du plant et est attachée à son autre extrémité à une petite masse qui permet de maintenir la ficelle tendue. Lorsque la jeune feuille s'allonge, la petite masse descend et fait tourner la poulie. La croissance de la feuille peut alors être mesurée en "tours de poulie" via un petit dispositif électronique (en encodeur rotatif) qui suit mécaniquement le mouvement de la poulie.

On commence par importer les données du fichier CSV de manière similaire à précédemment pour les données des balances :

In [ ]:
df = pd.read_csv('data/croissance-2025.csv', sep=';')

# Conversion de la colonne 'time' en objet datetime
df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d %H:%M:%S UTC')

# Conversion d'UTC vers notre fuseau horaire (+1h)
df['time'] += pd.Timedelta('01:00:00')

# On utilise la colonne 'time' comme indice du DataFrame
df = df.set_index('time')

# Affichage des 5 premières lignes
df.head()

Que des "0" dans les 5 premières lignes du DataFrame. Pour s'assurer qu'il contient bien quelque chose, on peut utilise la fonction `tail()` qui affiche les 5 dernières lignes du DataFrame :

In [ ]:
df.tail()

C'est bon, le DataFrame contient bien quelque chose. Par contre, la colonne `pot_5` ne contient que des 0. On la supprime :

In [ ]:
df = df.drop(columns=['enc_5'])

Maintenant, c'est à vous de jouer !

### Question 1

Les données enregistrées par les encodeurs rotatifs ne correspondent pas *directement* à des longueurs de feuilles. Chaque variation de $\pm 1$ enregistrée par un encodeur rotatif correspond à un 1/80e de tour de poulie. Sachant que le diamètre de la poulie est de 2,6 cm, convertissez les données des encodeurs rotatifs en centimètres d'élongation des feuilles. Affichez le résultat sur un graphique, en fonction du temps.

*Hint* :
- le signe des valeurs mesurées par les encodeurs rotatifs traduit le sens de rotation de la poulie. Pensez-vous que cette information soit pertinente ici ?
- `from math import pi` vous permettra d'utiliser la variable `pi` dans votre code.

#### Correction

D'abord, le sens de rotation de la poulie n'a pas d'intérêt ici. On utilise la fonction `abs()` (valeur absolue) sur l'ensemble des données pour s'assurer de ne plus traiter que des valeurs positives par la suite :

In [ ]:
df = abs(df)

# Affichage des 5 dernières lignes pour voir ce qu'on fait
df.tail()

Ensuite, on convertit les "unités potentiomètres" (1/80e de tour de poulie d'un diamètre de 2,6 cm) en centimètres :

In [ ]:
from math import pi

df = df * 1/80 * pi * 2.6

# Affichage des 5 dernières lignes pour voir ce qu'on fait
df.tail()

Enfin, on affiche les données sur un graphique, en fonction du temps :

In [ ]:
df.plot(grid=True,
        xticks=np.unique(df.index.date),
        ylabel="[cm]")

### Question 2

Les discontinuités visibles sur le graphe précédent sont dues aux manipulations de vos chers assistants lorsque la ficelle est transférée d'une feuille à l'autre et que la masse est remontée de quelques dizaines de centimètres.

Sur des graphiques différents, affichez la courbe de croissance mesurée par les encodeurs n°4 et n°6 entre le 28-01-2025 et le 03-02-2025 (1e graphique) et entre le 05-02-2025 et le 11-02-2025 (2e graphique). Qu'observez-vous ?

#### Correction

On utilise encore la fonction `plot()`, mais cette fois en indiquant la colonne et l'intervalle de temps à afficher. Rappelez-vous que le format de date à utiliser est `YYYY:MM:DD`.

In [ ]:
df.loc['2025-01-28':'2025-02-03', ['enc_4', 'enc_6']].plot(grid=True)

In [ ]:
df.loc['2025-02-05':'2025-02-11', ['enc_4', 'enc_6']].plot(grid=True)

### Question 3

Affichez maintenant la croissance mesurée par l'encodeur n°6 entre du 12 février 15h30 au 14 février 15h30. Qu'observez-vous ? Par quel(s) mécanisme(s) pouvez-vous expliquer cette dynamique journalière ?


#### Correction

In [ ]:
df.loc['2025-02-12 16:30':'2025-02-14 16:30', 'enc_6'].plot(grid=True)

## Bonus : les données du poromètre

Comme précédemment, on commence par lire les données contenues dans le fichier CSV et on convertit les dates et heures en objet `datetime` pour faciliter leur manipulation par la suite :

In [ ]:
import pandas as pd

df = pd.read_csv('data/porometre-2025.csv', sep=';', encoding='latin-1')

# Concaténation des colonnes 'date' et 'heure' et conversion en datetime via to_datetime()
df['time'] = df['date'] + ' ' + df['heure']
df['time'] = pd.to_datetime(df['time'], format='%d-%m-%y %H:%M')

# Les colonnes 'date' et 'heure' sont remplacées par l'unique colonne 'time', on les supprime donc du DataFrame
df = df.drop(columns=['date', 'heure'])

# On affiche les 5 premières lignes du DataFrame
df.head()

### Question 1

A l'aide de graphique, montrez l'influence (ou non) d'une variable au choix (date ou heure de la mesure, rang de la feuille, face de la feuille, position sur la feuille, état de la feuille ou PAR) sur la conductance stomatique.

*Hint* :
- la fonction `df.groupby()` permet par exemple de calculer la moyenne de la conductance stomatique mesurée sur les faces abaxiales et adaxiales respectivement : ```df.groupby('face_f')['cond'].mean()``` ;
- pour explorer les données, la fonction `describe()` est intéressante. Par exemple, ```df.groupby('face_f')['cond'].describe()``` ;
- la fonction `df.scatter(x=<abscisses des points à afficher>, y=<ordonnées des points à afficher>)` permet d'afficher un "[scatter plot](https://en.wikipedia.org/wiki/Scatter_plot)" (ou "nuage de points" en français) ;
- la fonction `df.plot(column=<la colonne à afficher>, by=<les différentes catégories>)` permet d'afficher un "[box plot](https://fr.wikipedia.org/wiki/Bo%C3%AEte_%C3%A0_moustaches)" (ou "boîte à moustaches" en français).

### Question 2

Faites des hypothèses sur les raisons physiologiques expliquant l'influence (ou non) de cette variable sur la conductance stomatique.